In [ ]:
import pymysql
import pandas as pd

# Configuración de la base de datos
DB_CONFIG = {
    "host": "localhost",  # Cambia según tu configuración
    "user": "root",       # Usuario de MySQL
    "password": "",       # Contraseña (déjalo vacío si no tiene)
    "database": "karlaortiz_datos1",  # Nombre de la base de datos
    "port": 3306           # Puerto de MySQL (por defecto es 3306)
}

# Cargar el archivo CSV
csv_file = "modified_codigos_postales.csv"
df = pd.read_csv(csv_file, encoding="utf-8", delimiter=",")

# Reemplazar NaN con NULL
df = df.where(pd.notna(df), None)

# Crear la conexión a MySQL
conn = pymysql.connect(**DB_CONFIG)
cursor = conn.cursor()

# Query para insertar datos
def insert_data(row):
    sql = """
    INSERT INTO codigos_postales (
        id, d_codigo, d_asenta, d_tipo_asenta, d_mnpio, d_estado,
        d_CP, c_estado, c_oficina, c_tipo_asenta, c_mnpio,
        id_asenta_cpcons, d_zona, seccion_electoral
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, tuple(row))

# Insertar cada fila
for _, row in df.iterrows():
    insert_data(row)

# Confirmar cambios y cerrar conexión
conn.commit()
cursor.close()
conn.close()

print("Importación completada exitosamente.")
c


Note: you may need to restart the kernel to use updated packages.


In [7]:
import pymysql

# Configuración de la base de datos
DB_CONFIG = {
    "host": "162.210.102.177",  # Cambia según tu configuración
    "user": "karlaortiz_karlaortiz",       # Usuario de MySQL
    "password": "wagJ;azv@24+",       # Contraseña (déjalo vacío si no tiene)
    "database": "karlaortiz_datos1",  # Nombre de la base de datos
    "port": 3306           # Puerto de MySQL (por defecto es 3306)
}

# Comprobar la conexión a la base de datos
try:
    conn = pymysql.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute("SELECT VERSION()")
    version = cursor.fetchone()
    print(f"Conectado a MySQL, versión: {version[0]}")
    cursor.close()
    conn.close()
except pymysql.MySQLError as e:
    print(f"Error al conectar a MySQL: {e}")


Error al conectar a MySQL: (1045, "Access denied for user 'karlaortiz_karlaortiz'@'fixed-187-189-49-79.totalplay.net' (using password: YES)")


In [8]:
pip install fastapi uvicorn pandas


Note: you may need to restart the kernel to use updated packages.


In [9]:
from fastapi import FastAPI
import pandas as pd

# Cargar datos desde el archivo CSV
df = pd.read_csv("modified_codigos_postales.csv")

# Inicializar FastAPI
app = FastAPI()

@app.get("/colonias/{codigo_postal}")
def obtener_colonias(codigo_postal: int):
    # Filtrar datos por código postal
    colonias = df[df["d_codigo"] == codigo_postal]
    
    if colonias.empty:
        return {"mensaje": "Código postal no encontrado"}
    
    # Convertir resultados a lista de diccionarios
    resultado = colonias[["d_asenta", "d_tipo_asenta", "d_mnpio", "d_estado", "d_zona"]].to_dict(orient="records")
    return {"codigo_postal": codigo_postal, "colonias": resultado}

# Para ejecutar la API:
# uvicorn geonames_api:app --reload
